# Mesh generation for tutorial 05

In [ ]:
import pygmsh
from dolfinx import Mesh, MeshFunction, MeshValueCollection, MPI
from dolfinx.cpp.mesh import CellType, GhostMode
from dolfinx.io import XDMFFile
from dolfinx.plotting import plot

### Geometrical parameters

In [ ]:
r = 3
lcar = 1. / 4.

### Create mesh

In [ ]:
geom = pygmsh.built_in.Geometry()
c0 = geom.add_circle([0.0, 0.0, 0.0], r, lcar)
geom.add_physical(c0.plane_surface, label=0)
geom.add_physical(c0.line_loop.lines, label=1)
pygmsh_mesh = pygmsh.generate_mesh(geom)
mesh = Mesh(MPI.comm_world, CellType.triangle, pygmsh_mesh.points[:, :2],
            pygmsh_mesh.cells_dict["triangle"], [], GhostMode.none)

In [ ]:
plot(mesh)

### Create subdomains

In [ ]:
subdomains = MeshFunction("int", mesh, mesh.topology.dim, 0)

### Create boundaries

In [ ]:
boundaries_mvc = MeshValueCollection("int", mesh, mesh.topology.dim - 1, pygmsh_mesh.cells_dict["line"],
                                     pygmsh_mesh.cell_data_dict["gmsh:physical"]["line"])
boundaries = MeshFunction("int", mesh, boundaries_mvc, 0)

### Save

In [ ]:
with XDMFFile(MPI.comm_world, "circle.xdmf") as output:
    output.write(mesh)
with XDMFFile(MPI.comm_world, "circle_subdomains.xdmf") as output:
    output.write(subdomains)
with XDMFFile(MPI.comm_world, "circle_boundaries.xdmf") as output:
    output.write(boundaries)